# Initilaisation

In [ ]:
import ROOT
# from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

# Saclay studies

In [ ]:
# 0 - desy
# 1 - mm1
# 2 - er2
# 3 - er3

# 4 - DESY cosmic
path = '~/LXPLUS/DESY_testbeam/cosmic_strict_v2/'
# path = '/eos/user/s/ssuvorov/DESY_testbeam/cosmic_strict_v2/'
voltage = '380'

file = [ROOT.TFile(path + '../tree/g_' + voltage + '_200_dedx.root')]
file.append(ROOT.TFile(path + '/mm1_' + voltage + 'V_200ns_dedx.root'))
file.append(ROOT.TFile(path + '/er2_' + voltage + 'V_dedx.root'))
file.append(ROOT.TFile(path + '/er3_' + voltage + 'V_dedx.root'))

# file.append(ROOT.TFile('~/LXPLUS/DESY_testbeam/multi_pad/c_0d_0p2T_200ns_50Mhz_dedx.root'))
file.append(ROOT.TFile(path + '../tree/c_0d_0T_200ns_25Mhz_dedx.root'))

name = ['DESY beam', 'MM1', 'ERAM2', 'ERAM3', 'DESY_cosmic', 'DESY_cosmic 2']
color = [ROOT.kBlack, ROOT.kRed, ROOT.kGreen, ROOT.kBlue, ROOT.kViolet, ROOT.kCyan]

### Charge

In [ ]:
%jsroot on
cl.Clear()
cl.Divide(2, 2)

ROOT.gStyle.SetOptTitle(0)

hs = [ROOT.THStack("hs_charge", "")]
hs.append(ROOT.THStack("hs_dedx", ""));
hs.append(ROOT.THStack("hs_mult", ""))

x_axis = ['Untruncated charge [a.u.]', 'Truncated charge [a.u.]', 'Multiplicity']
x_axis_r = [[0., 5000.], [0., 2000.], [0., 10.]]

for i in [0, 1, 2, 3, 4]:
    charge = file[i].Get('un_trunc_cluster')
    charge.Scale(1./charge.Integral())
    charge.SetLineColor(color[i])
    charge.SetTitle(name[i])
    hs[0].Add(charge)


    dedx = file[i].Get('dEdx')
    dedx.Scale(1./dedx.Integral())
    dedx.SetLineColor(color[i])
    dedx.SetTitle(name[i])
    dedx.GetXaxis().SetRangeUser(0., 2000.)
    hs[1].Add(dedx)

    Mult = file[i].Get('Mult')
    Mult.Scale(1./Mult.Integral())
    Mult.SetLineColor(color[i])
    Mult.SetTitle(name[i])
    hs[2].Add(Mult)
    
for i, h in enumerate(hs):
    cl.cd(i+1)
    h.Draw('histo nostack')
    h.GetXaxis().SetTitle(x_axis[i])
    h.GetXaxis().SetRangeUser(x_axis_r[i][0], x_axis_r[i][1])
    ROOT.gPad.BuildLegend(0.5, 0.55, 0.8, 0.85)

    
cl.Draw()

### Charge fit

In [ ]:
c.Clear()
c.cd()
hs = ROOT.THStack("hs_charge", "")
for i in [0, 1, 2, 3, 4]:
    dedx = file[i].Get('dEdx')
    dedx.Scale(1./dedx.Integral())
    dedx.SetLineColor(color[i])
    dedx.SetTitle(name[i])
    dedx.GetXaxis().SetRangeUser(0., 2000.)
    hs.Add(dedx)
    dedx.Fit('gaus')
    sigma = dedx.GetFunction('gaus').GetParameter(2)
    mean = dedx.GetFunction('gaus').GetParameter(1)
    print(100*sigma/mean)
hs.Draw('histo nostack')
hs.GetXaxis().SetRangeUser(0., 1600) 
c.Draw()

### Charge in different pads in the cluster

In [ ]:
%jsroot on
cw.Clear()
cw.cd()
cw.Divide(3)
ROOT.gStyle.SetOptTitle(1)

hs = []
for pad in range(3):
    cw.cd(pad+1)
    hs.append(ROOT.THStack('hs', ''))
    h = []
    for i in [0, 1, 2, 3]:
        if pad == 0:
            h.append(ROOT.TH1F('h', '', 100, 0., 2000.))
        else:
            h.append(ROOT.TH1F('h', '', 200, 0., 800.))
            
        file[i].outtree.Project('h', f'pad_charge[{pad}][]', f'multiplicity > {pad+1}')
        h[-1].Scale(1./h[-1].Integral())
        h[-1].SetTitle(name[i])
        h[-1].SetLineColor(color[i])
        hs[-1].Add(h[-1])
    
    hs[-1].Draw('histo nostack')
    hs[-1].GetXaxis().SetTitle('Charge [a.u.]')
    hs[-1].SetTitle(f'Pad {pad+1}')
    ROOT.gPad.BuildLegend()
cw.Draw()    

## Angular distributions

In [ ]:
cw.Clear()
cw.Divide(2)

cw.cd(1)
f_id = 1
angle = file[f_id].Get('angle')
angle.GetXaxis().SetRangeUser(0., 0.4)
angle.GetYaxis().SetRangeUser(0., 0.4)
angle.SetTitle(name[f_id])
angle.GetXaxis().SetTitle('tg(#phi)')
angle.GetYaxis().SetTitle('tg(#theta)')
angle.Draw('colz')

cw.cd(2)
f_id = 2
angle = file[f_id].Get('angle')
angle.GetXaxis().SetRangeUser(0., 0.4)
angle.GetYaxis().SetRangeUser(0., 0.4)
angle.SetTitle(name[f_id])
angle.GetXaxis().SetTitle('tg(#phi)')
angle.GetYaxis().SetTitle('tg(#theta)')
angle.Draw('colz')

cw.Draw()

## Gas quality

In [ ]:
%jsroot on
cl.Clear()
cl.cd()
cl.Divide(2, 2)

h2 = []

for i in [0, 1, 2, 3]:
    cl.cd(i+1)
    h2.append(ROOT.TH2F('h2', '', 511, 0., 511., 100, 0., 3000.))
    file[i].outtree.Project('h2', 'dEdx:pad_time[0][]', '', '')
    h2[i].GetXaxis().SetTitle('t_{1}')
    h2[i].GetYaxis().SetTitle('Cluster charge')
    h2[i].Draw('colz')
    
cl.Draw()

In [ ]:
%jsroot on
c.Clear()
c.cd()

h2 = []

hs = ROOT.THStack('hs', '')

for i in [1, 2, 3]:
#     cl.cd(i+1)
    h2.append(ROOT.TH1F('h2', '', 511, 0., 511.))
    file[i].outtree.Project('h2', 'pad_time[0][]', '', '')
    h2[-1].Scale(1./h2[-1].Integral())
    h2[-1].SetLineColor(color[i])
    h2[-1].SetTitle(name[i])
    hs.Add(h2[-1])

hs.Draw('histo nostack')
hs.GetXaxis().SetTitle('t_{1}')
c.BuildLegend()   
c.Draw()

In [ ]:
%jsroot on
cl.Clear()
cl.cd()
cl.Divide(2, 2)

for i in [0, 1, 2, 3]:
    cl.cd(i+1)
    h2 = ROOT.TH2F('h2', '', 511, 0., 511., 100, 0., 3000.)
    file[i].outtree.Project('h2', 'dEdx:pad_time[0][]', '', '')
    h2.GetXaxis().SetTitle('t_{1}')
    h2.GetYaxis().SetTitle('Cluster charge')
    h2.Draw('colz')
    
cl.Draw()

## Multiplicities

In [ ]:
path = '~/LXPLUS/DESY_testbeam/tree_v2/'

file = [ROOT.TFile(path + 'z_all_360_200_02T.root')]
file.append(ROOT.TFile(path + '/z_all_360_200_0T.root'))
# file = [ROOT.TFile(path + '/z_360_275_200_02T_430.root')]
# file.append(ROOT.TFile(path + '/z_360_275_200_0T_430.root'))

title = ['.2T', '0T']

In [ ]:
%jsroot on
cw.cd()
cw.Clear()
cw.Divide(2)

h2 = []
ROOT.gStyle.SetOptTitle(1)
for i, f in enumerate(file):
    h2.append(ROOT.TH2F('h', '', 36, 0., 36., 10, 0., 10.))
    f.outtree.Project('h', 'multiplicity:colID', 'multiplicity > -1')
    cw.cd(i+1)
    h2[-1].SetTitle(title[i])
    h2[-1].Draw('colz')
    
cw.Draw()

In [ ]:
%jsroot on
cl.cd()
cl.Clear()
cl.Divide(2, 2)

h2_hm = []
h2_lm = []
h2_hm_dedx = []
h2_lm_dedx = []
hs = []
COI = 1
diff_2 = []
diff_2_e = []
diff_0 = []
diff_0_e = []
ROOT.gStyle.SetOptTitle(1)
for COI_id in range(1, 2):
    for i, f in enumerate(file):
        h2_hm.append(ROOT.TH1F('h_hm', '', 10, 0., 10.))
        h2_lm.append(ROOT.TH1F('h_lm', '', 10, 0., 10.))
        h2_hm_dedx.append(ROOT.TH1F('h_hm_dedx', '', 100, 0., 2000.))
        h2_lm_dedx.append(ROOT.TH1F('h_lm_dedx', '', 100, 0., 2000.))
        for entry in f.outtree:
            for col in range(len(entry.colID)):
                if entry.colID[col] == COI_id:
                    first_mult = entry.multiplicity[col]
                    continue

            if first_mult >= 3:
                h2_hm_dedx[-1].Fill(entry.dEdx)
            else:
                h2_lm_dedx[-1].Fill(entry.dEdx)

            for col in range(len(entry.colID)):
                col_id = entry.colID[col]
                mult = entry.multiplicity[col]
                if col_id == -999:
                    continue
                if first_mult >= 3:
                    h2_hm[-1].Fill(mult)
                else:
                    h2_lm[-1].Fill(mult)

        h2_hm[-1].SetLineColor(ROOT.kRed)
        h2_hm[-1].Scale(1./h2_hm[-1].Integral())
        h2_hm[-1].SetTitle(f"mult {COI} col >= 3")

        h2_lm[-1].SetLineColor(ROOT.kBlack)
        h2_lm[-1].Scale(1./h2_lm[-1].Integral())
        h2_lm[-1].SetTitle(f"mult {COI} col < 3")

        h2_hm_dedx[-1].SetLineColor(ROOT.kRed)
        h2_hm_dedx[-1].Scale(1./h2_hm_dedx[-1].Integral())
        h2_hm_dedx[-1].SetTitle(f"mult {COI} col >= 3")

        h2_lm_dedx[-1].SetLineColor(ROOT.kBlack)
        h2_lm_dedx[-1].Scale(1./h2_lm_dedx[-1].Integral())
        h2_lm_dedx[-1].SetTitle(f"mult {COI} col < 3")



        cl.cd(i+1)
        hs.append(ROOT.THStack())
        hs[-1].Add(h2_hm[-1])
        hs[-1].Add(h2_lm[-1])
        hs[-1].Draw('nostack histo')
        hs[-1].SetTitle(title[i])
        ROOT.gPad.BuildLegend()

        cl.cd(i+3)
        hs.append(ROOT.THStack())
        hs[-1].Add(h2_hm_dedx[-1])
        hs[-1].Add(h2_lm_dedx[-1])
        hs[-1].Draw('nostack histo')
        hs[-1].SetTitle(title[i])
        ROOT.gPad.BuildLegend()
        c.cd()
        h2_hm_dedx[-1].Fit('gaus', 'Q', '', 800, 1600)
        h2_lm_dedx[-1].Fit('gaus', 'Q', '', 800, 1600)
        s = h2_hm_dedx[-1].GetFunction('gaus').GetParameter(2)
        m = h2_hm_dedx[-1].GetFunction('gaus').GetParameter(1)
        s_e = h2_hm_dedx[-1].GetFunction('gaus').GetParError(2)
        m_e = h2_hm_dedx[-1].GetFunction('gaus').GetParError(1)
        res_h = 100*s/m
        res_h_e = 100*s/m * ((s_e/s)**2 + (m_e/m)**2)**0.5
#         print(f'{title[i]}\t mult {COI} col >= 3', str(format(res_h, '.2f')) + '% +- ' + str(format(res_h_e, '.2f')),sep='\t')
        s = h2_lm_dedx[-1].GetFunction('gaus').GetParameter(2)
        m = h2_lm_dedx[-1].GetFunction('gaus').GetParameter(1)
        s_e = h2_lm_dedx[-1].GetFunction('gaus').GetParError(2)
        m_e = h2_lm_dedx[-1].GetFunction('gaus').GetParError(1)
        res_l = 100*s/m
        res_l_e = 100*s/m * ((s_e/s)**2 + (m_e/m)**2)**0.5
#         print(f'{title[i]}\t mult {COI} col < 3', str(format(res_l, '.2f')) + '% +- ' + str(format(res_l_e, '.2f')),sep='\t')
        print('\r' + str(COI_id), end='')
        if i == 0:
            diff_2.append(res_h - res_l)
            diff_2_e.append((res_h - res_l) * (res_h_e**2 + res_l_e**2) **0.5)
        else:
            diff_0.append(res_h - res_l)
            diff_0_e.append((res_h - res_l) * (res_h_e**2 + res_l_e**2) **0.5)
        cl.cd()

cl.cd()    
cl.Draw()

In [ ]:
c.cd()
mg = ROOT.TMultiGraph()
gr_0 = ROOT.TGraphErrors()
for i in range(len(diff_0)):
    gr_0.SetPoint(gr_0.GetN(), i+1, diff_0[i])
    gr_0.SetPointError(gr_0.GetN()-1, 0, abs(diff_0_e[i]))
    
gr_2 = ROOT.TGraphErrors()
for i in range(len(diff_2)):
    gr_2.SetPoint(gr_2.GetN(), i+1, diff_2[i])
    gr_2.SetPointError(gr_2.GetN()-1, 0, abs(diff_2_e[i]))
    
gr_2.SetMarkerColor(ROOT.kRed)
gr_2.SetTitle('.2T')
gr_0.SetTitle('.0T')
mg.Add(gr_0)
mg.Add(gr_2)
mg.Draw('ap')
mg.GetYaxis().SetTitle("resol. high m. - resol. low m.")
mg.GetXaxis().SetTitle("column to guess high/low mult")
c.Draw()
c.BuildLegend()

## Pad sum truncation

In [ ]:
%jsroot on

# file = [ROOT.TFile(path + '/z_360_275_200_02T_430.root')]
# file.append(ROOT.TFile(path + '/z_360_275_200_0T_430.root'))

cl.cd()
cl.Clear()
cl.Divide(2, 2)

cl.cd(1)
hs = [ROOT.THStack()]
h_0T = [ROOT.TH1F('h_0T', '0T', 10, 0., 10.)]
file[1].outtree.Project('h_0T', 'multiplicity', 'multiplicity > 0')
h_0T[-1].Scale(1./h_0T[-1].Integral())
hs[-1].Add(h_0T[-1])

h_2T = [ROOT.TH1F('h_2T', '.2T', 10, 0., 10.)]
file[0].outtree.Project('h_2T', 'multiplicity', 'multiplicity > 0')
h_2T[-1].Scale(1./h_2T[-1].Integral())
h_2T[-1].SetLineColor(ROOT.kRed)
hs[-1].Add(h_2T[-1])
hs[-1].Draw('histo nostack')
ROOT.gPad.BuildLegend()

cl.cd(2)
hs.append(ROOT.THStack())
h_0T.append(ROOT.TH1F('h_0Ta', '0T', 100, 0., 1000.))
file[1].outtree.Project('h_0Ta', 'pad_charge[1][]', 'pad_charge[1][] > 0')
h_0T[-1].Scale(1./h_0T[-1].Integral())
hs[-1].Add(h_0T[-1])

h_2T.append(ROOT.TH1F('h_2Tb', '.2T', 100, 0., 1000.))
file[0].outtree.Project('h_2Tb', 'pad_charge[1][]', 'pad_charge[1][] > 0')
h_2T[-1].Scale(1./h_2T[-1].Integral())
h_2T[-1].SetLineColor(ROOT.kRed)
hs[-1].Add(h_2T[-1])
hs[-1].Draw('histo nostack')
ROOT.gPad.BuildLegend()

cl.cd(3)
trunc = .7
pad_used = 9

h_0T_2d = ROOT.TH2F('h_0Tb_2d', '.0T', 100, 500., 3000., 100, 0., 1000.)
h_2T_2d = ROOT.TH2F('h_2Tb_2d', '.2T', 100, 500., 3000., 100, 0., 1000.)

hs.append(ROOT.THStack())
h_0T.append(ROOT.TH1F('h_0T', '0T', 100, 0., 2000.))
for entry in file[1].outtree:
    dedx_1 = []
    dedx_2 = []
    for i in range(1, 35):
        sum_1 = 0
        sum_2 = 0
#         if entry.pad_charge[i + 36*0] > 0:
#             sum_1 = entry.pad_charge[i + 36*0]
        if entry.pad_charge[i + 36*1] > 0:
            sum_2 = entry.pad_charge[i + 36*1]
        for j in range(0, pad_used):
            q = entry.pad_charge[i + 36*j]
            if q > 0:
                sum_1 += q
        if sum_1 != 0:
            dedx_1.append(sum_1)
        if sum_2 != 0:
            dedx_2.append(sum_2)
    dedx_1.sort()
    dedx_2.sort()
    av_1 = 0
    av_2 = 0
    for i in range(int(trunc*len(dedx_1))):
        av_1 += dedx_1[i] / int(trunc*len(dedx_1))
    for i in range(int(trunc*len(dedx_2))):
        av_2 += dedx_2[i] / int(trunc*len(dedx_2))
    h_0T[-1].Fill(av_1)
    h_0T_2d.Fill(av_1, av_2)

h_0T[-1].Scale(1./h_0T[-1].Integral())

h_2T.append(ROOT.TH1F('h_2T', '.2T', 100, 0., 2000.))

for entry in file[0].outtree:
    dedx_1 = []
    dedx_2 = []
    for i in range(1, 35):
        sum_1 = 0
        sum_2 = 0
#         if entry.pad_charge[i + 36*0] > 0:
#             sum_1 = entry.pad_charge[i + 36*0]
        if entry.pad_charge[i + 36*1] > 0:
            sum_2 = entry.pad_charge[i + 36*1]
        sum_2 = 0
        for j in range(0, pad_used):
            q = entry.pad_charge[i + 36*j]
            if q > 0:
                sum_1 += q
        if sum_1 != 0:
            dedx_1.append(sum_1)
        if sum_2 != 0:
            dedx_2.append(sum_2)
    dedx_1.sort()
    dedx_2.sort()
    av_1 = 0
    av_2 = 0
    for i in range(int(trunc*len(dedx_1))):
        av_1 += dedx_1[i] / int(trunc*len(dedx_1))
    for i in range(int(trunc*len(dedx_2))):
        av_2 += dedx_2[i] / int(trunc*len(dedx_2))
    h_2T[-1].Fill(av_1)
    h_2T_2d.Fill(av_1, av_2)
    
h_2T[-1].Scale(1./h_2T[-1].Integral())
# h_2T[-1].SetLineColor(ROOT.kRed)
hs[-1].Add(h_0T[-1])
# hs[-1].Add(h_2T[-1])

h_0T[-1].GetXaxis().SetTitle('Truncated mean [a.u.]')
h_0T[-1].Draw('hist')
ROOT.gPad.SetGrid()

cl.cd(4)
h_2T[-1].GetXaxis().SetTitle('Truncated mean [a.u.]')
h_2T[-1].Draw('hist')
ROOT.gPad.SetGrid()

cl.Draw()

In [ ]:
hs = ROOT.THStack()
h0 = ROOT.TH1F('h_02', '.2T', 100, 0., 0.05)
h01 = ROOT.TH1F('h_00', '0T', 100, 0., 0.05)

file[0].outtree.Project('h_02', 'abs(angle_xy)', 'angle_xy > -10')
file[1].outtree.Project('h_00', 'abs(angle_xy)', 'angle_xy > -10')

h0.Scale(1./h0.Integral())
h01.Scale(1./h01.Integral())
h0.SetLineColor(ROOT.kRed)

hs.Add(h0)
hs.Add(h01)

c.cd()
hs.Draw('histo nostack')
c.Draw()
c.BuildLegend()

## dE/dx for different slope

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)

pad_used = 1
trunc = .7

h_l = []
h_a = []

hs = []

for i in range(2):
    h_l.append(ROOT.TH1F('h_0Tl', f'{title[i]} straight', 100, 0., 3000.))
    h_a.append(ROOT.TH1F('h_0Ta', f'{title[i]} sloped', 100, 0., 3000.))
    for entry in file[i].outtree:
        dedx_1 = []
        for col in range(1, 35):
            sum_1 = 0
            for j in range(0, pad_used):
                q = entry.pad_charge[col + 36*j]
                if q > 0:
                    sum_1 += q
            if sum_1 != 0:
                dedx_1.append(sum_1)
        dedx_1.sort()
        av_1 = 0
        for it in range(int(trunc*len(dedx_1))):
            av_1 += dedx_1[it] / int(trunc*len(dedx_1))
        if entry.pad_y[1] == entry.pad_y[34]:
            h_l[-1].Fill(av_1)
        if entry.pad_y[1] == entry.pad_y[34] - 1:
            h_a[-1].Fill(av_1)

    hs.append(ROOT.THStack())
    h_a[-1].SetLineColor(ROOT.kRed)
#     h_a[-1].Scale(1./h_a[-1].Integral())
#     h_l[-1].Scale(1./h_l[-1].Integral())
    hs[-1].Add(h_a[-1])
    hs[-1].Add(h_l[-1])
    
#     h_a[-1].Fit('gaus', 'Q', '', 800, 1700)
#     s = h_a[-1].GetFunction('gaus').GetParameter(2)
#     m = h_a[-1].GetFunction('gaus').GetParameter(1)
#     print(title[i], 'sloped', 100*s/m, sep = '\t')
#     h_l[-1].Fit('gaus', 'Q', '', 800, 1700)
#     s = h_l[-1].GetFunction('gaus').GetParameter(2)
#     m = h_l[-1].GetFunction('gaus').GetParameter(1)
#     print(title[i], 'straight', 100*s/m, sep = '\t')

    cw.cd(i+1)
    hs[-1].Draw('histo nostack')
    ROOT.gPad.BuildLegend()
cw.Draw()

In [ ]:
c.cd()
h = ROOT.TH1F('h', '', 100, 0., 1.)
file[0].outtree.Project('h', '1.*pad_charge[0][] / charge[]', 'pad_charge[0][] > 0 && pad_y[0][1] == pad_y[0][34] -1 && pad_y[0][1] != -999')
h.Draw()
c.Draw()

## dE/dx over track position

In [ ]:
%jsroot on
c.cd()

file = ROOT.TFile(path + 'z_all_360_200_02T_SR.root')

h2 = ROOT.TH2F('h', '', 100, 0., 2000., 100, -20, 20)
h = ROOT.TH1F('h_0T', '0T', 100, 0., 2000.)

trunc = .7

mean_cut = 0.001
rms_cut = 0.007

mean_dy = 0
m_n = 0

for entry in file.outtree:
    summ = 0
    n = 0
    mn = 99999
    mx = -9999
    for col in range(0, 70):
        dx = entry.track_pos[col] 
        if dx == -999:
            continue;
        if dx > mx:
            mx = dx
        if dx < mn:
            mn = dx
        summ += dx
        n += 1
    if (n == 0):
        continue
    if mx-mn <0.1:
        mean_dy += mx - mn
        m_n += 1
    mean = summ / n
    if abs(mean) > 0.001:
        continue
    dedx_1 = []
    for col in range(0, 70):
        q = entry.charge[col]
        if q > 0:
            dedx_1.append(q)
    dedx_1.sort()
    av_1 = 0
    for it in range(int(trunc*len(dedx_1))):
        av_1 += dedx_1[it] / int(trunc*len(dedx_1))
    h2.Fill(av_1, mean*1e3)
    h.Fill(av_1)
    
# h2.GetXaxis().SetTitle('Truncated mean [a.u.]')
# h2.GetYaxis().SetTitle('Mean track position [mm]')
# h2.Draw('colz')
h.Draw()
h.Fit('gaus', 'Q', '', 800, 1700)
s = h.GetFunction('gaus').GetParameter(2)
m = h.GetFunction('gaus').GetParameter(1)
print('0T', 'sloped', 100*s/m, sep = '\t')
print(mean_dy / m_n)
c.SetGrid()
c.Draw()